# ChatDoc:和文件聊天

In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("openai.env")
os.environ["OPENAI_API_KEY"] = "sk-jxKghxq3MMF3aC1J5c75666dD3D04e2dBaB36eE75b72Dd89"
os.environ["OPENAI_API_BASE"] = "https://ai-yyds.com/v1"

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
print("OPENAI_API_KEY:", api_key)

OPENAI_API_KEY: sk-jxKghxq3MMF3aC1J5c75666dD3D04e2dBaB36eE75b72Dd89


In [3]:
import os

# 设置HTTP代理
os.environ['HTTP_PROXY'] = "127.0.0.1:7890"
# 设置HTTPS代理
os.environ['HTTPS_PROXY'] = "127.0.0.1:7890"

# 验证设置
print("HTTP_PROXY:", os.environ.get('HTTP_PROXY'))
print("HTTPS_PROXY:", os.environ.get('HTTPS_PROXY'))


HTTP_PROXY: 127.0.0.1:7890
HTTPS_PROXY: 127.0.0.1:7890


In [4]:
#导入必须的包
from langchain.document_loaders import UnstructuredExcelLoader,Docx2txtLoader,PyPDFLoader
from langchain.text_splitter import  CharacterTextSplitter
from langchain.embeddings import  OpenAIEmbeddings
from langchain.vectorstores import  Chroma
#导入聊天所需的模块
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

import openai


class ChatDoc():
    def __init__(self):
        self.doc = None
        self.splitText = [] #分割后的文本
        self.template = [
            ("system","你是一个处理文档的秘书,你从不说自己是一个大模型或者AI助手,你会根据下面提供的上下文内容来继续回答问题.\n 上下文内容\n {context} \n"),
            ("human","你好！"),
            ("ai","你好"),
            ("human","{question}"),
        ]
        self.prompt = ChatPromptTemplate.from_messages(self.template)
    
    def getFile(self):
        doc = self.doc
        loaders = {
            "docx":Docx2txtLoader,
            "pdf":PyPDFLoader,
            "xlsx":UnstructuredExcelLoader,
        }

        file_extension = doc.split(".")[-1]
        loader_class = loaders.get(file_extension)
        if loader_class:
            try:
                loader = loader_class(doc)
                return loader.load()
            except Exception as e:
                print(f"Error loading {file_extension} files:{e}")
        else:
            print(f"Unsupported file extension: {file_extension}")
            return None 

    #处理文档的函数
    def splitSentences(self):
        full_text = self.getFile() #获取文档内容
        if full_text != None:
            #对文档进行分割
            text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
            self.splitText = text_splitter.split_documents(full_text)

    #向量化与向量存储
    def embeddingAndVectorDB(self):
        embeddings = OpenAIEmbeddings()
        db = Chroma.from_documents(
            documents = self.splitText,
            embedding = embeddings
        )
        return db
    
    #提问并找到相关的文本块
    def askAndFindFiles(self, question):
        db = self.embeddingAndVectorDB()
        #retriever = db.as_retriever(search_type="mmr")
        retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold":0.5, "k":1})
        return retriever.get_relevant_documents(query=question)
    
    #用自然语言和文档聊天
    def chatWithDoc(self,question):
        _content = ""
        content = self.askAndFindFiles(question)
        for i in content:
            _content += i.page_content
        
        message = self.prompt.format_messages(context=_content, question=question)
        chat = ChatOpenAI(
            model="gpt-4",
            temperature=0
        )
        return chat.invoke(message)

chat_doc = ChatDoc()
chat_doc.doc = "./example/fake.docx"
chat_doc.splitSentences()
chat_doc.chatWithDoc("公司注册地址是哪里？")

c:\Users\hudon\.conda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


AIMessage(content='公司的注册地址是江苏省南京市雨花台区软件大道101号。', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 258, 'total_tokens': 283}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-35162a1a-93e9-457c-a86c-2ab201e6ae23-0')